In [ ]:
from os import listdir, path
import matplotlib.pyplot as plt
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from keras.models import Sequential, Model
from tqdm import tqdm
from IPython.display import FileLink
from tensorflow import keras

In [ ]:
PATH = 'dataset/'

In [ ]:
print('Total No of yoga poses in the dataset is:', len(listdir(PATH)))

In [ ]:
poses_in_dataset = listdir(PATH)

In [ ]:
classes = listdir(PATH)
class_names_label = {class_name:i for i, class_name in enumerate(classes)}

In [ ]:
labels = []
for folder in listdir(PATH):
    label = class_names_label[folder]
    for file in tqdm(listdir(path.join(PATH, folder))):
        labels.append(label)

In [ ]:
batch_size = 32
img_height = 512
img_width = 512

In [ ]:
training = keras.preprocessing.image_dataset_from_directory(
  PATH,
  validation_split=0.25,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
validation = keras.preprocessing.image_dataset_from_directory(
  PATH,
  validation_split=0.25,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = training.class_names

In [ ]:
normalization_layer = keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(512, 512, 3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [ ]:
BASE=keras.applications.EfficientNetB3(include_top=False, weights="imagenet"
               ,input_shape=(img_height, img_width, 3), pooling='max')
x = BASE.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(107, activation="softmax")(x)
model = Model(BASE.input, predictions)
# model = keras.models.Sequential([
#     keras.layers.Conv2D(64, (3,3), activation='relu',padding = 'Same', input_shape=(224, 224, 3)),
#     keras.layers.MaxPooling2D(2, 2),
#     keras.layers.Dropout(0.25),
#     keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
#     keras.layers.MaxPooling2D(2,2),
#     keras.layers.Dropout(0.25),
#     keras.layers.Conv2D(256, (3,3), activation='relu',padding = 'Same'),
#     keras.layers.MaxPooling2D(2,2),
#     keras.layers.Dropout(0.25),
#     keras.layers.Flatten(),
#     keras.layers.Dense(1024, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(5, activation='softmax')
# ])

In [ ]:
model.compile(
  optimizer='adam',
  loss = 'sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [ ]:
history = model.fit(training, validation_data=validation, epochs=100)

In [ ]:
def plotLearningCurve(history,epochs):
  epochRange = range(1,epochs+1)
  plt.figure(figsize = (12,6))
  plt.plot(epochRange,history.history['accuracy'])
  plt.plot(epochRange,history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend(['Train','Validation'],loc='upper left')
  plt.show()

  plt.figure(figsize = (12,6))
  plt.plot(epochRange,history.history['loss'])
  plt.plot(epochRange,history.history['val_loss'])
  plt.title('Model Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend(['Train','Validation'],loc='upper left')
  plt.show()

In [ ]:
plotLearningCurve(history, 100)

In [ ]:
total_image_count = []
for i in range(107):
    nestest_path = path + poses_in_dataset[i]
    total_image_count.append(len(os.listdir(nestest_path)))
total_image_count
plt.figure(figsize = (16,6))
plt.xticks(rotation = 90)
plt.bar(poses_in_dataset, len(total_image_count))

In [ ]:
model.save('posetrainer.h5')
FileLink(r'posetrainer.h5')

In [ ]:
testing_model = keras.models.load_model('posetrainer.h5')